In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.method as mem
import numpy as np

***通过随机函数生成测试数据，用于后续检验函数调用示例***

In [2]:
fo = np.random.rand(1000,10)
error = np.random.rand(1000,10)-0.5
ob = fo + error
ob[ob>0.5] = 1
ob[ob<=0.5] = 0

### bs评分   
**bs(Ob, Fo)**  
计算brier_score 评分
  
**参数说明：**  
 Ob:实况数据, 任意维numpy数组，元素取值必须为0或者1   
 Fo:预测数据, 任意维numpy数组,Fo.shape 和Ob.shape一致，元素取值必须在区间[0,1]内    
 return: 0到1的实数，完美值为0  
 **调用示例：** 

In [3]:
mem.bs(ob,fo)

0.170657803432625

### bss评分   
**bss(Ob, Fo)**  
计算Brier skill score  评分, 它是实际预报bs相对预报随机预报bs评分的技巧。
  
**参数说明：**  
 Ob:实况数据, 任意维numpy数组，元素取值必须为0或者1   
 Fo:预测数据, 任意维numpy数组,Fo.shape 和Ob.shape一致，元素取值必须在区间[0,1]内    
 return: 负无穷到1的实数，完美值为1  
 **调用示例：** 

In [4]:
mem.bss(ob,fo)

0.3173626425332827

### roc面积   
**roc_auc(Ob, Fo)**  
计算roc图中 roc曲线以下部分的面积。
  
**参数说明：**  
 Ob:实况数据, 任意维numpy数组，元素取值必须为0或者1   
 Fo:预测数据, 任意维numpy数组,Fo.shape 和Ob.shape一致，元素取值必须在区间[0,1]内    
 return: 0 到 1的实数，完美值为1，小于或等于0.5代表没有预报技巧。  
 **调用示例：** 

In [5]:
mem.roc_auc(ob,fo)

0.8255361299064035

   在以上示例中，观测和预报的数据都直接输入到评分函数中进行计算，然而有些情况下待检验的数据太大不能整体存入一个numpy数组中，或者不方便整体存入一个numpy数组中，此时就不能调用上面的方式调用评分函数。为此本程序库设计了一套可分块计算的检验程序。
其检验步骤如下：
步骤1：根据需要将分块数据逐一输入到中间结果计算函数
步骤2：将中间结果进行累加或合并
步骤3：根据累加或合并的中间结果计算检验指标
通常上述计算中步骤1是最耗费计算资源，为了提高效率步骤1也可以采用并行的方式执行。此外，步骤1执行的结果也可输出到文件中，在后续的检验可以从中读入部分中间结果执行后续步骤，从而可以实现各种方式的分组检验，大大提高检验计算效率。  
   另外，在预报检验经常需要进行分组检验，获得不同类别预报的评分指标并进行对比。此时可以应用上述基于中间结果的检验函数对多维中间统计量的整体计算能力来简化代码的复杂度。

### 总样本数、误差总和、观测平均、观测方差   
**tems(Ob,Fo)**  
用来计算bs,bss等检验指标的中间量   
   
**参数说明：**  
 Ob:实况数据，任意维numpy数组  
 Fo:预测数据，任意维numpy数组,Fo.shape 和Ob.shape一致  
 return: 长度为4的一维numpy数组，其内容依次为总样本数、误差总和、观测的平均值、观测的方差  
   

### tems合并函数
**tems_merge(tems0,tems1)**  
用来将tems计算得到的分块中间统计量进行合并，得到两块数据并集对应的统计量  
**参数说明：**  
 tems0: 长度4的一维数组，分别记录了（总样本数、误差总和、观测的平均值、观测的方差）  
 tems1: 长度4的一维数组，分别记录了（总样本数、误差总和、观测的平均值、观测的方差）  
 return: 长度4的一维数组，分别记录了（总样本数、误差总和、观测的平均值、观测的方差）  
   
**调用示例**  

In [7]:
model_count = 5
day_count = 100
tems_array = np.zeros((model_count,4))  
grade_count = 10
hnh_array = np.zeros((model_count,grade_count,2))
for i in range(day_count):
    ob = np.random.rand(1000)
    ob[ob>0.5] = 1
    ob[ob<=0.5] = 0
    for j in range(model_count):
        fo = np.random.rand(1000)
        hnh_array[j,:,:] += mem.hnh(ob,fo,grade_count) #计算并累加区分能力表，函数见probablility.plot.hnh模块
        tems1 = mem.tems(ob,fo)
        tems_array[j,:] = mem.tems_merge(tems_array[j,:],tems1)

### bs评分   
**bs_tems(tems_array)**  
计算brier_score 评分
  
**参数说明：**  
 tems_array:多维numpy数组,其最后一维长度为4，其内容依次为总样本数、误差总和、观测的平均值、观测的方差    
 return: numpy数组，其中每个元素为 0到1的实数，完美值为0  
 **调用示例：** 

In [8]:
mem.bs_tems(tems_array)

array([0.33463285, 0.33228084, 0.33398223, 0.33182809, 0.33342913])

### bss评分   
**bss_tems(tems_array)**  
计算Brier skill score  评分, 它是实际预报bs相对预报随机预报bs评分的技巧。
  
**参数说明：**  
 tems_array:多维numpy数组,其最后一维长度为4，其内容依次为总样本数、误差总和、观测的平均值、观测的方差   
 return: numpy数组，其中每个元素为负无穷到1的实数，完美值为1  
 **调用示例：** 

In [9]:
mem.bss_tems(tems_array)

array([-0.33855154, -0.32914336, -0.33594903, -0.32733233, -0.33373661])

### roc面积   
**roc_auc_hnh(hnh_array)**  
计算roc图中 roc曲线以下部分的面积。
  
**参数说明：**  
 hnh_array: grade_count × 2 的numpy数组，其中元素[i,0]和[i,1]分别为预报概率在第i个区间，观测发生和不发生的样本数     
 Fo:预测数据 预测数据 任意维numpy数组,Fo.shape 和Ob.shape一致，元素取值必须在区间[0,1]内    
 return: numpy数组，其中每个元素为0 到 1的实数，完美值为1，小于或等于0.5代表没有预报技巧。  
 **调用示例：** 

In [10]:
mem.roc_auc_hnh(hnh_array)

array([0.49724141, 0.50172634, 0.4991554 , 0.50251744, 0.49947603])

以上只是展示了分类检验的维度为1的情况，实际上上述思路可以扩展至任意高维的情况。熟练使用中间统计量计算和合并方法，基于中间统计量整体计算分类问题下的检验指标数组，是提高代码编写效果的关键。上述检验函数的内部也都采用了numpy的整体计算方式实现，在计算效率上进行了最大程度的优化。